# Longwave Radiation Transfer Example

This notebook demonstrates how to use the PyRTE-RRTMGP package to solve a longwave radiative transfer problem using the `pyrte_rrtmgp` library. Specifically:

- It downloads the required data
- Computes the gas optics using an absorption approach
- Solves the radiation transfer equations
- Validates the computed fluxes against reference datasets

See the [documentation](https://pyrte-rrtmgp.readthedocs.io/en/latest/) for more information.

In [ ]:
import os
import numpy as np
import xarray as xr
import dask

## Importing pyrte_rrtmgp Modules

We import the following modules from `pyrte_rrtmgp`:

- `rrtmgp_gas_optics`: Handles spectral properties of gases
- `rrtmgp_data`: Manages reference data
- `rte_solver`: Solves the radiative transfer equation

In [ ]:
from pyrte_rrtmgp import rrtmgp_gas_optics
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver

## Data Download and Directory Setup

Download the RRTMGP data and set up the directory paths for input and reference files.

In [ ]:
rte_rrtmgp_dir = download_rrtmgp_data()
rfmip_dir = os.path.join(rte_rrtmgp_dir, "examples", "rfmip-clear-sky")
input_dir = os.path.join(rfmip_dir, "inputs")
ref_dir = os.path.join(rfmip_dir, "reference")
os.listdir(rte_rrtmgp_dir)

## Loading Gas Optics and Atmosphere Dataset

Load the longwave **gas optics file** (`LW_G256`) and the corresponding atmosphere dataset for the simulation.

The gas optics file contains 256 g-points (`LW_G256`), which provides a balance between accuracy and computational efficiency. G-points refer to the spectral quadrature points used in the correlated-k distribution method for radiative transfer.

We also load the **atmospheric data** from the [RFMIP (Radiative Forcing Model Intercomparison Project)](https://rfmip.leeds.ac.uk/) dataset. We then open this data with [xarray](https://docs.xarray.dev) and [Dask](https://www.dask.org/) to enable parallel processing. Parallel processing with Dask is automatically active because we provide the `chunks` argument to the `open_dataset` function. Chunking happens along the experiment dimension `expt`.

In [ ]:
gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)

atmosphere_file = "multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
atmosphere_path = os.path.join(input_dir, atmosphere_file)
atmosphere = xr.open_dataset(atmosphere_path, chunks={"expt": 3})
atmosphere

## Computing Gas Optics and Solving Radiation Transfer Equations

We compute the gas optics using an absorption method and solve the longwave radiation transfer equations to obtain the fluxes:

- The `gas_optics.compute()` method calculates the absorption properties for each atmospheric layer based on its composition and thermodynamic state.
- The `solve()` method from `RTESolver` computes the radiative fluxes using the gas optics and atmospheric data.

In [ ]:
gas_optics_lw.gas_optics.compute(atmosphere, problem_type="absorption")

solver = RTESolver()
fluxes = solver.solve(atmosphere, add_to_input=False)
fluxes

## Validating Computed Fluxes Against Reference Data

Load the reference datasets for upward (`rlu`) and downward (`rld`) longwave fluxes and validate that the computed fluxes match within the specified error tolerance (`ERROR_TOLERANCE = 1e-7`).

In [ ]:
ERROR_TOLERANCE = 1e-7

rlu_reference = f"{ref_dir}/rlu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rld_reference = f"{ref_dir}/rld_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rlu = xr.load_dataset(rlu_reference, decode_cf=False)
rld = xr.load_dataset(rld_reference, decode_cf=False)

assert np.isclose(
    fluxes["lw_flux_up"], rlu["rlu"], atol=ERROR_TOLERANCE
).all(), "Longwave flux up mismatch"
assert np.isclose(
    fluxes["lw_flux_down"], rld["rld"], atol=ERROR_TOLERANCE
).all(), "Longwave flux down mismatch"

print("Longwave radiation transfer calculations validated successfully!")